# -*- coding: utf-8 -*-
"""Spring2024_Streaming_Word_Count_with_Apache_Spark_Streaming.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1kfEMsZrlHged3z4gbn9yznFHDvi5QGMI
"""
### Streaming Word Count with Apache Spark Streaming
**This project focuses on real-time word counting using Apache Spark Streaming, a powerful framework for processing and analyzing streaming data. In this scenario, we aim to continuously process incoming text data streams, perform word counting, and visualize the results in real-time. Here's an overview of the functionalities and testing scenarios for this project:**

Functionality Overview:
1. Data Streaming: Apache Spark Streaming ingests data streams from a source, such as a file system, in mini-batches, allowing for continuous processing.
2. Word Counting: The streaming application tokenizes the text data, removes stop words, and calculates the frequency of each word in the stream.
3. Real-time Visualization: The word counts are visualized dynamically, enabling users to observe changes in word frequencies as new data arrives.
4. Continuous Processing: The streaming application operates indefinitely, processing incoming data streams in real-time without interruption.

Testing Rate Limiting:
To test rate limiting, we can simulate a high influx of text data by continuously feeding the streaming application with a large volume of text. We can monitor the application's behavior to ensure that it handles the incoming data stream efficiently without overwhelming the system. Additionally, we can introduce delays or throttling mechanisms to observe how the application responds to varying data rates.

Database Change Monitoring:
While Apache Spark Streaming is primarily designed for processing real-time data streams, we can integrate it with database monitoring systems to track changes in underlying data sources. For example, we can monitor updates to a database table containing text data and trigger the streaming application to reprocess the affected data streams accordingly. This ensures that the streaming application remains synchronized with the underlying data source and reflects any changes in real-time.

Additional Features:
We can enhance the streaming word count application by incorporating additional features such as fault tolerance, windowed computations, stateful processing, and integration with external systems for data ingestion and output. Experimenting with these features allows us to explore the full capabilities of Apache Spark Streaming and tailor the application to specific use cases and requirements.

By testing these functionalities and scenarios, we can gain valuable insights into the behavior and performance of the streaming word count application, identify potential bottlenecks or limitations, and fine-tune its configuration to optimize performance and reliability in real-world deployment scenarios.

**In this section, to run the streaming word count application with Apache Spark and visualize the results, we need to install the required Python libraries. This section of the code installs two essential dependencies:**

1. **pyspark**: This library provides the Python API for Apache Spark, allowing us to interact with Spark's distributed computing engine and perform data processing tasks.

2. **matplotlib**: This library is used for data visualization in Python. We'll use it to create visualizations of the word counts generated by the streaming application.

pip install pyspark
pip install matplotlib

**In the below section of the code, we import a collection of Python libraries essential for the execution of the streaming word count application using Apache Spark. These libraries provide various functionalities crucial for data processing, manipulation, visualization, and interaction with the Spark framework. They include "pyspark" for accessing Spark's distributed computing engine, "sys" for system-specific parameters and functionalities, "operator" for efficient operations on iterable data structures, "matplotlib.pyplot" for creating visualizations, "pyspark.sql.functions" for working with structured data in Spark SQL, "SparkSession" for initializing the connection to the Spark cluster, "Tokenizer" and "StopWordsRemover" from the "pyspark.ml.feature" module for text preprocessing, "SparkContext" for creating Resilient Distributed Datasets (RDDs), and "StreamingContext" for handling streaming functionality in Spark. Each library serves a unique purpose in facilitating the development and execution of the streaming word count application, ensuring efficient data processing and visualization within the Spark framework.**
import pyspark

# Required imports
import sys
from operator import add
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

**The function below, the process_stream function is defined to handle streaming data and generate word counts in near real-time. It takes an RDD as input, extracts words from the data, counts their occurrences, sorts them, and displays the top words along with their counts. Additionally, it visualizes the top 10 words using a bar plot, providing a quick and efficient way to analyze word frequencies in the streaming data.**
# Function to process streaming data and generate word counts
def process_stream(rdd):
    if not rdd.isEmpty():
        word_counts = rdd.withColumn('word', F.explode(F.col('words_clean'))) \
                         .groupBy('word') \
                         .count() \
                         .sort('count', ascending=False)

        # Show the result
        word_counts.show()

        # Visualize word counts
        word_counts_pd = word_counts.limit(10).toPandas()  # Limit to top 10 words for visualization
        plt.figure(figsize=(10, 6))
        plt.bar(word_counts_pd['word'], word_counts_pd['count'])
        plt.xlabel('Words')
        plt.ylabel('Count')
        plt.title('Top 10 Words in Billboard Songs')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

**The code below the SparkContext (sc) and StreamingContext (ssc) objects are initialized to set up the environment for real-time streaming processing. The SparkContext serves as the entry point to the Spark cluster, while the StreamingContext facilitates the creation of DStream objects, enabling the application to process continuous streams of data with a micro-batch interval of 5 seconds.**
# Initialize SparkContext and StreamingContext
sc = SparkContext(appName="RealTimeWordCount")
ssc = StreamingContext(sc, 5)  # 5-second micro-batch interval

**In the section below, the SparkSession object (spark) is initialized to enable the use of Spark SQL and the DataFrame API for processing structured data. By creating a unified entry point for interacting with different data sources and executing SQL queries, SparkSession facilitates seamless data manipulation within the Spark application, with the application name set to "PythonWordCount".**
# Initialize SparkSession
spark = SparkSession.builder.appName("PythonWordCount").getOrCreate()

**Below, the code reads CSV data from the file 'billboard_lyrics_1964-2015.csv' into a Spark DataFrame called data and then prints the extracted data with column headers.**
# Read CSV data
data = spark.read.format('csv').options(header='true', inferSchema='true') \
    .load('billboard_lyrics_1964-2015.csv')
print('############ CSV extract:')
data.show()

**The below function we tokenize the lyrics data from the DataFrame using the Tokenizer class from the pyspark.ml.feature module. The Tokenizer splits the text into individual words or tokens, creating a new column named "words_token" containing the tokenized lyrics. The resulting DataFrame, containing the original rank along with the tokenized lyrics, is then displayed to provide a glimpse of the tokenized data.**
tokenizer = Tokenizer(inputCol="Lyrics", outputCol="words_token")
tokenized = tokenizer.transform(data).select('Rank', 'words_token')

print('############ Tokenized data extract:')
tokenized.show(truncate=False)

**In this section below, we remove stop words from the tokenized lyrics data using the StopWordsRemover class from the pyspark.ml.feature module. Stop words, common words like "the" or "and", are filtered out, resulting in a cleaner representation of the lyrics. The DataFrame containing the rank and cleaned words is displayed to provide insight into the cleaned data.**
# Remove stop words
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
data_clean = remover.transform(tokenized).select('Rank', 'words_clean')

print('############ Data Cleaning extract:')
data_clean.show(truncate=False)

**In this final word count section, we compute the frequency of each word in the cleaned lyrics data. The DataFrame is transformed to explode the cleaned words column into individual rows, grouped by word, and then aggregated to count the occurrences of each word. The result is sorted in descending order based on word count, providing insights into the most common words in the lyrics dataset.**
# Final word count
result = data_clean.withColumn('word', F.explode(F.col('words_clean'))) \
                   .groupBy('word') \
                   .count().sort('count', ascending=False)

print('############ Final word count:')
result.show()

**In this section, we install the Pandas library using pip and import it as pd for easy access.**
pip install pandas
import pandas as pd

**The below visualization is the word counts obtained from the streaming data analysis. The word counts DataFrame (result) is converted to a Pandas DataFrame (result_pd) to facilitate visualization. Using Matplotlib, we create a bar plot to display the top 10 words and their respective counts. The plot provides a visual representation of the most frequently occurring words in the streaming data, aiding in data interpretation and analysis**
# Tokenize and remove stop words
import matplotlib.pyplot as plt
words = [row['word'] for row in result.limit(10).collect()]
counts = [row['count'] for row in result.limit(10).collect()]

# Create a bar plot using matplotlib
plt.figure(figsize=(10, 6))
plt.bar(words, counts)
plt.xlabel('Words')
plt.ylabel('Count')
plt.title('Top 10 Words in Billboard Songs')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**The below function, we create a DStream (Discretized Stream) by reading text files from a specified directory. The textFileStream method of the StreamingContext (ssc) is used to create the stream, with the directory path provided as the argument. This enables the streaming application to ingest and process data from the text files in real-time, allowing for continuous analysis of the data as it becomes available.**
# Create DStream by reading text files from the directory
data_dir = "./data_chunks"
stream = ssc.textFileStream(data_dir)

**Followed by the below code, we define a process to handle each RDD (Resilient Distributed Dataset) in the stream. The foreachRDD method is used to apply the process_stream function to each RDD in the stream. This function processes the streaming data, extracting word counts, and visualizing the results. By iterating over each RDD in the stream, we ensure that the processing logic is applied to every batch of data received, enabling real-time analysis of the streaming data.**
# Process each RDD in the stream
stream.foreachRDD(process_stream)

**In the below process, we initiate the streaming process by calling the start method on the StreamingContext object (ssc). Additionally, we use the awaitTermination method to instruct the program to wait until the streaming process is terminated or manually stopped.**
# Start streaming
ssc.start()
# Wait for streaming to finish
ssc.awaitTermination()